In [143]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [145]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")

In [146]:
from datetime import datetime

from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2023, 9, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(36980, 674)
(36980,)
(20984, 674)
(20984,)


In [147]:
X_train

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
0,5,0,0,0,0,0,7,0,0,0,...,0,0,18,0,0,0,0,0,2023-01-29 00:00:00,HB101
1,24,0,0,0,0,0,7,0,0,0,...,0,0,3,0,0,0,0,0,2023-01-29 12:00:00,HB101
2,2,0,0,0,0,0,13,0,0,0,...,0,0,13,0,0,0,0,0,2023-01-30 00:00:00,HB101
3,34,0,0,0,0,0,18,0,0,0,...,0,0,17,0,0,0,0,0,2023-01-30 12:00:00,HB101
4,1,0,0,0,0,0,11,0,0,0,...,0,0,24,0,0,0,0,0,2023-01-31 00:00:00,HB101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36975,36,0,0,0,0,0,24,0,0,0,...,0,0,1,0,0,0,0,0,2023-08-29 12:00:00,JC116
36976,1,0,0,0,0,0,10,0,0,0,...,0,0,27,0,0,0,0,0,2023-08-30 00:00:00,JC116
36977,32,0,0,0,0,0,22,0,0,0,...,0,0,5,0,0,0,0,0,2023-08-30 12:00:00,JC116
36978,3,0,0,0,0,0,13,0,0,0,...,0,0,24,0,0,0,0,0,2023-08-31 00:00:00,JC116


In [148]:
#from src.pipeline_utils import get_pipeline
#pipeline = get_pipeline()

In [149]:
#pipeline

In [150]:
# Get your pipeline
from src.pipeline_utils import get_pipeline
pipeline = get_pipeline()

# Train
pipeline.fit(X_train, y_train)

# Predict
y_pred = pipeline.predict(X_test)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019620 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10690
[LightGBM] [Info] Number of data points in the train set: 36980, number of used features: 115
[LightGBM] [Info] Start training from score 6.905057


In [151]:
y_pred

array([ 0.69948779, 44.2803079 ,  5.29287255, ..., 15.19298491,
        2.03246091, 11.08187414])

In [152]:
from sklearn.metrics import mean_absolute_error

In [153]:
test_mae = mean_absolute_error(y_test, y_pred)
print(f"{test_mae:.4f}")

2.6619


In [154]:

import src.config as config

In [155]:
import joblib  


# Save the pipeline  
joblib.dump(pipeline, config.MODELS_DIR / "lgb_model.pkl")

['C:\\Users\\2003j\\Desktop\\citibike_ride_prediction\\models\\lgb_model.pkl']

In [156]:
import hopsworks

project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME, api_key_value=config.HOPSWORKS_API_KEY
)

feature_store = project.get_feature_store()
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION
)

2025-05-10 06:14:01,977 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 06:14:02,040 INFO: Initializing external client
2025-05-10 06:14:02,042 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 06:14:03,049 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214691


In [157]:
# Create a feature view if it doesn't already exist
try:
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all(),
    )
    print(f"Feature view '{config.FEATURE_VIEW_NAME}' (version {config.FEATURE_VIEW_VERSION}) created successfully.")
except Exception as e:
    print(f"Error creating feature view: {e}")

# Retrieve the feature view
try:
    feature_view = feature_store.get_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
    )
    print(f"Feature view '{config.FEATURE_VIEW_NAME}' (version {config.FEATURE_VIEW_VERSION}) retrieved successfully.")
except Exception as e:
    print(f"Error retrieving feature view: {e}")

Error creating feature view: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1214691/featurestores/1202327/featureview). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":270180,"usrMsg":"Queries must have features","errorMsg":"Cannot create feature view."}', error code: 270180, error msg: Cannot create feature view., user msg: Queries must have features
Feature view 'time_series_hourly_feature_view_citi_bike' (version 1) retrieved successfully.


In [158]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [159]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name=config.MODEL_NAME,
    metrics={"test_mae": test_mae},
    description="LightGBM model for bike rides",
    input_example=X_train.sample(),
    model_schema=model_schema,
    #include_files=["src/", "requirements.txt"]  # Include directories and files
)
# https://community.hopsworks.ai/t/attributeerror-windowspath-object-has-no-attribute-startswith/1003
model.save(str(config.MODELS_DIR) + "/lgb_model.pkl")
# model.save('C:\\Users\\mkhaw\\github\\sp25_taxi\\models\\lgb_model.pkl')

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/312107 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/2075 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/51419 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1214691/models/model_demand_predictor_next6hours/6


Model(name: 'model_demand_predictor_next6hours', version: 6)

In [160]:
model

Model(name: 'model_demand_predictor_next6hours', version: 6)

In [161]:
models = model_registry.get_models(name=config.MODEL_NAME)

In [162]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-05-10 06:14:22,171 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 06:14:22,191 INFO: Initializing external client
2025-05-10 06:14:22,193 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 06:14:23,417 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214691


In [163]:
model

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000002106B4D8820>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor', LGBMRegressor())])

In [164]:
from datetime import datetime, timedelta

import pandas as pd
import pytz
import src.config as config
from src.data_utils import transform_ts_data_info_features_and_target
from src.inference import (
    get_feature_store
)

# Get the current datetime64[us, Etc/UTC]
# for number in range(22, 24 * 29):
# current_date = pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=number)
est = pytz.timezone("America/New_York")

# Get current timestamp in EST
current_date = pd.Timestamp.now(tz=est)
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1 * 29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

2025-05-10 06:14:26,459 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 06:14:26,474 INFO: Initializing external client
2025-05-10 06:14:26,476 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-10 06:14:27,742 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214691
Fetching data from 2025-04-11 06:14:26.458346-04:00 to 2025-05-10 05:14:26.458346-04:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.02s) 


In [165]:
ts_data

,pickup_hour,pickup_location_id,rides


In [166]:
from src.inference import (
    get_feature_store,
    get_model_predictions,
    load_model_from_registry,
)

In [167]:
from src.data_utils import transform_ts_data_info_features

features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=12)

model = load_model_from_registry()

predictions = get_model_predictions(model, features)
predictions["pickup_hour"] = current_date.ceil("h")
print(predictions)

ValueError: No data could be transformed. Check if input DataFrame is empty or window size is too large.

In [ ]:


feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

feature_group.insert(predictions, write_options={"wait_for_job": False})


2025-05-09 23:51:24,250 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-09 23:51:24,254 INFO: Initializing external client
2025-05-09 23:51:24,254 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-09 23:51:24,860 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213637
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213637/fs/1201260/fg/1454547


Uploading Dataframe: 100.00% |██████████| Rows 83/83 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: bike_6hours_model_prediction_citibike_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213637/jobs/named/bike_6hours_model_prediction_citibike_1_offline_fg_materialization/executions


(Job('bike_6hours_model_prediction_citibike_1_offline_fg_materialization', 'SPARK'),
 None)